In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### Testing table extraction for the 1st page of the file

In [3]:
file ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/TX_2013.pdf'

In [4]:
pdf = pdfplumber.open(file)
table=pdf.pages[0].extract_table()
df_test = pd.DataFrame(table[1::],columns=table[0])

In [5]:
df_test.head()

,County,s\ne\nh\ns\na\nr\nC\n \nal\nt\na\nF,s\ne\nti\nali\nt\na\nF,s\nu\no\nri\nes\nSe\ned ash\nectCr\np\ns\nu\nS,s\nu\no\nri\nes\nSe\nd ri\neu\nctnj\neI\np\ns\nu\nS,g\nn\nti\na\ncites\nah\nps\naa\ncr\nnC\nI\n-\nn\no\nN,g\nn\nti\na\nt\npaciries\nau\ncnj\nnI\nI\n-\nn\no\nN,y\nr\nu\nnjes\n Ih\nes\nbla\nsiCr\ns\no\nP,s\ne\nri\nu\nnj\nI\n \ne\nbl\nsi\ns\no\nP,s\ne\nh\ns\na\nr\nC\n \ny\nr\nu\nnj\nI\n-\nn\no\nN,s\ne\nri\nu\nnj\nI\n-\nn\no\nN,y\nt\nri\ne\nSevhes\nn s\nwa\nr\noC\nn\nk\nn\nU,s\ne\nri\nu\nnj\nI\n \n \nn\nw\no\nn\nk\nn\nU,s\ne\nh\ns\na\nr\nC\n \nal\nt\no\nT
0,Anderson,9,9,15,20,160,212,107,183,485,"1,278",17,48,793
1,Andrews,10,12,8,10,41,57,46,70,181,506,8,21,294
2,Angelina,17,19,37,48,232,331,296,484,"1,202","3,673",47,151,"1,831"
3,Aransas,3,4,23,29,28,42,33,60,197,590,15,38,299
4,Archer,7,7,7,10,18,24,17,34,81,160,2,2,132


### Extracting all the tables from a PDF file

In [6]:
#Create df from table on first page to act as the first df:
pdf = pdfplumber.open(file)
pages = pdf.pages
tbl = pages[0].extract_table()
original_df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])

#Append data from remaining tables/pages:
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for i,pg in enumerate(pages):
        df = pd.DataFrame(tbl[1::]) 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes']) 
        original_df=original_df.merge(df, how='outer')      

In [7]:
original_df

,County,fatal_crashes,fatalities,suspected_serious_crashes,suspected_serious_injuries,non_incapacitating_crashes,non_incapacitating_injuries,possible_injury_crashes,possible_injuries,non_injury_crashes,non_injuries,unknown_severity_crashes,unknown_injuries,total_crashes
0,Anderson,9,9,15,20,160,212,107,183,485,"1,278",17,48,793
1,Andrews,10,12,8,10,41,57,46,70,181,506,8,21,294
2,Angelina,17,19,37,48,232,331,296,484,"1,202","3,673",47,151,"1,831"
3,Aransas,3,4,23,29,28,42,33,60,197,590,15,38,299
4,Archer,7,7,7,10,18,24,17,34,81,160,2,2,132
5,Armstrong,5,6,4,6,5,6,9,11,41,114,1,2,65
6,Atascosa,7,7,25,35,120,176,130,187,632,"1,755",19,49,933
7,Austin,6,6,26,37,45,65,62,86,370,"1,011",6,19,515
8,Bailey,2,2,1,2,14,22,18,36,56,151,2,6,93
9,Bandera,7,10,27,38,67,95,27,36,158,364,1,7,287


### Importing poppulation data and merging both files

In [8]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/pop_est_2010_2019_tx.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Anderson,57960,57829,57639,57526,58175,57974,57735
1,Andrews,16779,17435,18072,17791,17603,18011,18705
2,Angelina,87312,87567,87829,87682,87572,86810,86715
3,Aransas,23889,24566,24800,25162,25392,23446,23510
4,Archer,8801,8844,8763,8781,8783,8700,8553


In [9]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/pop_est_2020_2021_tx.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Anderson,57881,58402
1,Andrews,18612,18440
2,Angelina,86345,86506
3,Aransas,23916,24510
4,Archer,8557,8681


In [10]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Anderson,57960,57829,57639,57526,58175,57974,57735,57881,58402
1,Andrews,16779,17435,18072,17791,17603,18011,18705,18612,18440
2,Angelina,87312,87567,87829,87682,87572,86810,86715,86345,86506
3,Aransas,23889,24566,24800,25162,25392,23446,23510,23916,24510
4,Archer,8801,8844,8763,8781,8783,8700,8553,8557,8681


### Merging pop df and carsh data for speciffic year

In [11]:
var = 2013

In [12]:
pdf = pdfplumber.open(file)
pages = pdf.pages
tbl = pages[0].extract_table()
original_df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for i,pg in enumerate(pages):
        df = pd.DataFrame(tbl[1::]) 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes']) 
        original_df=original_df.merge(df, how='outer')      

In [13]:
original_df['suspected_serious_injuries'] =original_df['suspected_serious_injuries'].str.replace(',', '').astype(float)
original_df['possible_injuries'] =original_df['possible_injuries'].str.replace(',', '').astype(float)
original_df['injuries'] = original_df['suspected_serious_injuries'] + original_df['possible_injuries']
df_combined = pd.merge(original_df,pop_combined[['County', var]],on="County",how='left')
df_combined=df_combined.drop(['fatal_crashes','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','possible_injuries'], axis=1)
df_combined.head()

,County,fatalities,total_crashes,injuries,2013
0,Anderson,9,793,203.0,57960.0
1,Andrews,12,294,80.0,16779.0
2,Angelina,19,"1,831",532.0,87312.0
3,Aransas,4,299,89.0,23889.0
4,Archer,7,132,44.0,8801.0


In [14]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255 entries, 0 to 254
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   County         255 non-null    object 
 1   fatalities     255 non-null    object 
 2   total_crashes  255 non-null    object 
 3   injuries       255 non-null    float64
 4   2013           236 non-null    float64
dtypes: float64(2), object(3)
memory usage: 12.0+ KB


### Adding Metropolitans and Micropolitans

In [15]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tx = metros_df[metros_df['State Name']=='Texas']
metros_tx.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
3,"Palestine, TX",Micropolitan Statistical Area,Anderson,Texas
5,"Andrews, TX",Micropolitan Statistical Area,Andrews,Texas
6,"Lufkin, TX",Micropolitan Statistical Area,Angelina,Texas
8,"Rockport, TX",Micropolitan Statistical Area,Aransas,Texas
9,"Wichita Falls, TX",Metropolitan Statistical Area,Archer,Texas


In [16]:
df_combined_new = pd.merge(df_combined,metros_tx[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_new.head(20)

,County,fatalities,total_crashes,injuries,2013,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Anderson,9,793,203.0,57960.0,"Palestine, TX",Micropolitan Statistical Area
1,Andrews,12,294,80.0,16779.0,"Andrews, TX",Micropolitan Statistical Area
2,Angelina,19,"1,831",532.0,87312.0,"Lufkin, TX",Micropolitan Statistical Area
3,Aransas,4,299,89.0,23889.0,"Rockport, TX",Micropolitan Statistical Area
4,Archer,7,132,44.0,8801.0,"Wichita Falls, TX",Metropolitan Statistical Area
5,Armstrong,6,65,17.0,1940.0,"Amarillo, TX",Metropolitan Statistical Area
6,Atascosa,7,933,222.0,47021.0,"San Antonio-New Braunfels, TX",Metropolitan Statistical Area
7,Austin,6,515,123.0,28679.0,"Houston-The Woodlands-Sugar Land, TX",Metropolitan Statistical Area
8,Bailey,2,93,38.0,7124.0,NaN,NaN
9,Bandera,10,287,74.0,20557.0,"San Antonio-New Braunfels, TX",Metropolitan Statistical Area


### Adding everything together to create combined reports by year

In [19]:
for filename in os.listdir("C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/"):
    if filename.endswith(".pdf"):
       year = re.findall(r'\d{4}',filename)
       for var in year:
           name = var       
       pdf = pdfplumber.open(os.path.join("C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TX/", filename))
       tx_df = pd.DataFrame(columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
  'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
       pages = pdf.pages
       for i,pg in enumerate(pages): 
        tbl = pages[i].extract_table()
        df = pd.DataFrame(tbl[1::],columns=['County','fatal_crashes','fatalities','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','possible_injuries','non_injury_crashes',
'non_injuries','unknown_severity_crashes','unknown_injuries','total_crashes'])
        tx_df = tx_df.append(df) 
        if i== 9:
            name = int(name)
            tx_df['suspected_serious_injuries'] = tx_df['suspected_serious_injuries'].str.replace(',', '').astype(float)
            tx_df['possible_injuries'] = tx_df['possible_injuries'].str.replace(',', '').astype(float)
            tx_df['injuries'] = tx_df['suspected_serious_injuries'] + tx_df['possible_injuries']
            tx_df = tx_df.drop(['fatal_crashes','suspected_serious_crashes','suspected_serious_injuries','non_incapacitating_crashes','non_incapacitating_injuries','possible_injury_crashes','non_injury_crashes','non_injuries','unknown_severity_crashes','unknown_injuries','possible_injuries'], axis=1)                    
            tx_df = tx_df[['County','fatalities','injuries','total_crashes']]
            df_combined_tx = pd.merge(tx_df,pop_combined[['County', name]],on="County",how='left') 
            df_combined_tx_new = pd.merge(df_combined_tx,metros_tx[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tx_new.drop(index=df_combined_tx_new.index[-1],axis=0,inplace=True)
            df_combined_tx_new.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/%s_TX.xlsx' % name, engine='xlsxwriter',index=False)
        else:
            continue